In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from safep import parse_Colvars_log
from safep.RFEP_analysis import get_changing_bias, get_colvars_traj_filename, read_colvars_traj, print_TI_summary, get_precomputed_gradients, make_TI_figure

In [ ]:
# This should be the only user parameter we need
logfile = Path('RFEP_decouple.log')

# Everything is automated and generic below this

In [ ]:
global_conf, _, biases, TI_traj = parse_Colvars_log(logfile)

restraint = get_changing_bias(biases)
if restraint is None:
    raise ValueError("No changing bias found in the provided logfile.")
print(f"Processing TI data for restraint {restraint["name"]} on CVs {restraint["colvar"]}")

In [ ]:
# We assume the colvars traj and log are in the same directory
colvars_traj = get_colvars_traj_filename(global_conf, logfile.parent)
TI_cumulative, TI_per_window = read_colvars_traj(restraint, colvars_traj)
print_TI_summary(TI_cumulative)

In [ ]:
free_energy_gradients = get_precomputed_gradients(restraint, TI_traj)
fig, _ = make_TI_figure(TI_cumulative, TI_per_window, free_energy_gradients)
fig.savefig(Path(logfile).name.replace(".log", "_figures.png"))